# Get data

In [1]:
from utils.mt_5.data import get_currency_pair_data_

symbol = "GBPJPY"
lags_steps = 8

df = get_currency_pair_data_(symbol, timeframe='m15', years_back=.5)
df


,close
time,
2024-03-26 02:00:00,191.322
2024-03-26 02:15:00,191.293
2024-03-26 02:30:00,191.308
2024-03-26 02:45:00,191.289
2024-03-26 03:00:00,191.287
...,...
2024-12-05 18:30:00,191.610
2024-12-05 18:45:00,191.665
2024-12-05 19:00:00,191.632


# Learn serial dependence

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from utils.ts import make_lags, make_multistep_targets

X = make_lags(ts=df, lags=lags_steps).fillna(0.0)
y = make_multistep_targets(ts=df, steps=lags_steps).dropna()

X = X.iloc[lags_steps-1:].reset_index(drop=True)  
y = y.reset_index(drop=True) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = MultiOutputRegressor(XGBRegressor())
model.fit(X_train, y_train)

# # Room for Improvement

# `Add derived features` like rolling averages, RSI, or Bollinger Bands to capture additional information.
# `Optimize the XGBoost hyperparameters` using grid search or Bayesian optimization for potentially better accuracy.
# `Compare with other models` like LSTMs (for time series) or simple linear regression to benchmark performance.

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))

# Predict the future

In [3]:
last_lags = X.iloc[-1].values.reshape(1, -1)
y_fore = model.predict(last_lags) 

y_fore

array([[191.9911 , 191.76277, 191.53531, 191.61378, 191.68234, 191.63266,
        191.51486, 191.55316]], dtype=float32)

In [4]:
df.index[-1], df.iloc[-1].values[0]

(Timestamp('2024-12-05 19:30:00'), np.float64(191.46))

In [5]:
import pandas as pd

future_timestamps = pd.date_range(df.index[-1] + pd.Timedelta(minutes=15), periods=lags_steps, freq='15min')

forecast_df = pd.DataFrame({
    "time": future_timestamps,
    "predicted_close": y_fore.flatten()
})

forecast_df.index = forecast_df['time']
forecast_df.drop('time', axis=1, inplace=True)

print(f"\n\n......................forecast_df......................\n\n")
forecast_df



......................forecast_df......................




,predicted_close
time,
2024-12-05 19:45:00,191.991104
2024-12-05 20:00:00,191.762772
2024-12-05 20:15:00,191.535309
2024-12-05 20:30:00,191.613785
2024-12-05 20:45:00,191.682343
2024-12-05 21:00:00,191.632660
2024-12-05 21:15:00,191.514862
2024-12-05 21:30:00,191.553162


# Trade

In [8]:
current_price = df['close'].iloc[-1] 
predicted_close = float(f"{forecast_df['predicted_close'].iloc[-1]:.3f}")


if predicted_close > current_price: # buy
    target_profit = (predicted_close - current_price) * 100
    print(f".....................predicted_close is GREATER than current_close.....................\n")

if predicted_close < current_price: # short-sell
    target_profit = (current_price - predicted_close) * 100
    print(f"\n.....................We will go SHORT by {target_profit:.0f} pips.....................\n")

target_profit = int(float(f"{target_profit:.0f}"))

print(f"current_price   => {current_price}\npredicted_close => {predicted_close}\ntarget_profit   => {target_profit}")

.....................predicted_close is GREATER than current_close.....................

current_price   => 191.46
predicted_close => 191.553
target_profit   => 9


In [7]:
from utils.mt_5.trade import buy, short_sell

past_2000hrs = df.index.time[-1] >= pd.Timestamp('20:00:00').time()

if not past_2000hrs:
    if predicted_close > current_price: # buy
        print("\n\n.....................BUYING.....................\n\n")
        target_profit = round((predicted_close - current_price) * 100)
        buy(symbol, target_profit=target_profit)

    elif predicted_close < current_price: # short-sell
        print("\n\n......................SHORT-SELLING......................\n\n")
        target_profit = round((current_price - predicted_close) * 100)
        short_sell(symbol, target_profit=target_profit)
    else:
        print(".....................I DON'T KNOW HOW TO HELP YOU!.....................")
else:
    print("You CANNOT trade PAST 2000hrs\n\n.....................SELLING.....................\n\n")



.....................BUYING.....................


GBPJPYm already bought!
